In [5]:
from CGAN import CGAN
from utils import generate_augmentation, imshow_normalized, AugmentedDataset

from torchvision import datasets, transforms
from matplotlib import pyplot as plt

import numpy as np

from tqdm import tqdm
import torchmetrics
import torch
import torch.nn as nn
plt.gray()

<Figure size 640x480 with 0 Axes>

In [14]:
torch.cat((torch.tensor([[0],[0],[0]]), torch.tensor([[1,2],[1,2],[1,2]])), dim=1).tolist()


[[0, 1, 2], [0, 1, 2], [0, 1, 2]]

In [ ]:
class LOL(dict):
    def __init__(self, data):
        self.data = data
    def __getattr__(gan, attr):
        return gan.data[attr]

batch_size = 64
input_size = 28
num_classes = 47
do_train = False


args = LOL({
    "epoch": 10,
    "batch_size": 64,
    "save_dir": "models",
    "result_dir": "results",
    "dataset": "emnist",
    "log_dir": "logs",
    "gpu_mode": True,
    "gan_type": "CGAN",
    "input_size": 28,
    "lrG": 0.0002,
    "lrD": 0.0002,
    "beta1": 0.05,
    "beta2": 0.99,
    "class_num": 47,
})
### Load CGAN
try:
    gan = CGAN(args)
    gan.load()
except Exception as e:
    print("Couldn't load the model")
    do_train = True
    gan = CGAN(args)

---------- Networks architecture -------------
generator(
  (fc): Sequential(
    (0): Linear(in_features=109, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=6272, bias=True)
    (4): BatchNorm1d(6272, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (deconv): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): Tanh()
  )
)
Total number of parameters: 6688321
discriminator(
  (conv): Sequential(
    (0): Conv2d(48, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1,

In [ ]:
if do_train:
    gan.train()
else:
    print("already trained")

already trained


In [ ]:
input_size = 28
num_classes = 47

transform = transforms.Compose([transforms.Resize((input_size, input_size)), transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
original_trainset = datasets.EMNIST('data/emnist', train=True, download=True, transform=transform, split="bymerge")

counter = [0]*num_classes
for l in original_trainset.targets:
    k = int(l)
    counter[k] += 1
counter = np.array(counter)

### Number of augmentation per label
max_count = counter.max()
median = np.median(counter)
mean = counter.mean().round()
diffs = np.int32(mean - counter)
diffs[diffs < 0] = 0


In [ ]:
### The augmentation itself
data = torch.tensor([]).cuda()
targets = torch.tensor([]).cuda()
for label, diff in enumerate(diffs):
    if diff == 0:
        continue
    print(f"label: {label} count: {diff}")
    new = generate_augmentation(gan, label, diff)
    data = torch.cat([data, new])
    targets = torch.cat([targets, (torch.ones(diff)*label).cuda()])

targets = targets.cpu().to(torch.int64)
data = data.cpu().squeeze()
data = ((data + 1)*128).to(torch.uint8)
augmented_set = torch.cat((original_trainset.data, data))
augmented_targets = torch.cat((original_trainset.targets ,targets))
if augmented_set.shape[0] == original_trainset.data.shape[0] + data.shape[0]:
    print("concatenation of datasets is consistent")

label: 10 count: 8439
label: 11 count: 10976
label: 12 count: 1887
label: 13 count: 10244
label: 14 count: 9925
label: 15 count: 5752
label: 16 count: 12316
label: 17 count: 11753
label: 18 count: 117
label: 19 count: 9161
label: 20 count: 9852
label: 22 count: 3238
label: 23 count: 6613
label: 25 count: 4102
label: 26 count: 12247
label: 27 count: 9803
label: 29 count: 5084
label: 31 count: 7262
label: 32 count: 7447
label: 33 count: 9252
label: 34 count: 7758
label: 35 count: 9434
label: 36 count: 4841
label: 37 count: 9770
label: 38 count: 4698
label: 40 count: 12315
label: 41 count: 11157
label: 42 count: 6168
label: 43 count: 3406
label: 44 count: 11884
label: 45 count: 790
concatenation of datasets is consistent


In [ ]:
ext = "png"
augmented_set = augmented_set.to(torch.float)
for index, image in tqdm(enumerate(augmented_set)):
    fp = f"data/augmented/cgan/{augmented_targets[index]}/{index}.{ext}"
    utils.save_image(image, fp=fp, format=ext)

In [ ]:
with open("data/emnist/augmented/cgan.pkl", "wb") as f:
    torch.save(augmentd_set, f)